In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd

In [ ]:
# Mounting Google Drive to read Numpy file 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Loan and read Numpy file
bitfinex = np.load('/content/bitfinex_order_array.npy', allow_pickle=True)
bitfinex

array([{'orders': [[0.02701, 4, 19.63549352], [0.027, 4, 17.11867786], [0.02699, 3, 20.36402743], [0.02698, 2, 21.7853], [0.02697, 4, 43.29962648], [0.02696, 2, 7.45266557], [0.02695, 2, 4.90705179], [0.02694, 1, 5.6], [0.02693, 2, 42.37741964], [0.02692, 9, 13.4953], [0.02691, 11, 5.12], [0.0269, 12, 35.05647142], [0.02689, 12, 212.75016655], [0.02688, 14, 77.89796794], [0.02687, 13, 70.1598365], [0.02686, 12, 7.3613234], [0.02685, 13, 85.37315135], [0.02684, 11, 0.18682799], [0.02683, 13, 38.6210026], [0.02682, 11, 0.13620594], [0.02681, 11, 0.55056785], [0.0268, 12, 72.507], [0.02679, 11, 19.40053639], [0.02678, 12, 182.70556509], [0.02677, 10, 0.12], [0.02704, 6, -0.036], [0.02705, 11, -1.366348], [0.02706, 15, -12.38197914], [0.02707, 15, -37.09897034], [0.02708, 13, -13.14305331], [0.02709, 12, -27.79927846], [0.0271, 12, -22.84846225], [0.02711, 11, -25.06], [0.02712, 13, -60.23560576], [0.02713, 12, -34.309], [0.02714, 10, -0.06], [0.02715, 13, -53.70194438], [0.02716, 13, -69.

In [3]:
# Installing Python Spark
!pip install pyspark

     |████████████████████████████████| 204.2MB 74kB/s 
     |████████████████████████████████| 204kB 49.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=f94b811745963e21cad07659c5670f49bd3cc8bbd55d8a57eec729a15ce88187
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [4]:
# Importing necessary Spark libraries
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.ml.linalg import Vectors
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
#Making streaming for 10 seconds
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)
rdd = ssc.sparkContext.parallelize(bitfinex)

In [8]:
# Maping RDD 
rdd = rdd.map(lambda x: (x['orders'], x['time']))

In [9]:
rdd.take(1)

[([[0.02701, 4, 19.63549352],
   [0.027, 4, 17.11867786],
   [0.02699, 3, 20.36402743],
   [0.02698, 2, 21.7853],
   [0.02697, 4, 43.29962648],
   [0.02696, 2, 7.45266557],
   [0.02695, 2, 4.90705179],
   [0.02694, 1, 5.6],
   [0.02693, 2, 42.37741964],
   [0.02692, 9, 13.4953],
   [0.02691, 11, 5.12],
   [0.0269, 12, 35.05647142],
   [0.02689, 12, 212.75016655],
   [0.02688, 14, 77.89796794],
   [0.02687, 13, 70.1598365],
   [0.02686, 12, 7.3613234],
   [0.02685, 13, 85.37315135],
   [0.02684, 11, 0.18682799],
   [0.02683, 13, 38.6210026],
   [0.02682, 11, 0.13620594],
   [0.02681, 11, 0.55056785],
   [0.0268, 12, 72.507],
   [0.02679, 11, 19.40053639],
   [0.02678, 12, 182.70556509],
   [0.02677, 10, 0.12],
   [0.02704, 6, -0.036],
   [0.02705, 11, -1.366348],
   [0.02706, 15, -12.38197914],
   [0.02707, 15, -37.09897034],
   [0.02708, 13, -13.14305331],
   [0.02709, 12, -27.79927846],
   [0.0271, 12, -22.84846225],
   [0.02711, 11, -25.06],
   [0.02712, 13, -60.23560576],
   [0.0271

In [10]:
# Function to find max and min
def min_max(iterator):

    max_b = []
    min_a = []

    for x in iterator:

        bids = []
        asks = []


        for i in range(25):

            bids.append(x[0][i][0])
            asks.append(x[0][i+25][0])
          
        max_b.append([x[1],max(bids),min(asks)] )

    return max_b

In [11]:
final = rdd.mapPartitions(min_max)

In [12]:
# Displaying the final result
final.take(25)

[[1609195953321, 0.02701, 0.02704],
 [1609195954768, 0.02701, 0.02704],
 [1609195956195, 0.02702, 0.02704],
 [1609195957662, 0.02702, 0.02704],
 [1609195959134, 0.02703, 0.02704],
 [1609195960598, 0.02703, 0.02704],
 [1609195962124, 0.02706, 0.02709],
 [1609195963546, 0.02706, 0.02709],
 [1609195964951, 0.02707, 0.02709],
 [1609195966350, 0.02706, 0.02709],
 [1609195967728, 0.02706, 0.02709],
 [1609195969395, 0.02708, 0.02711],
 [1609195970821, 0.02708, 0.02711],
 [1609195972257, 0.02708, 0.02711],
 [1609195973669, 0.02708, 0.02711],
 [1609195975085, 0.02708, 0.02711],
 [1609195976495, 0.02707, 0.02711],
 [1609195978018, 0.02707, 0.02711],
 [1609195979445, 0.02706, 0.0271],
 [1609195980984, 0.02706, 0.0271],
 [1609195982389, 0.02706, 0.0271],
 [1609195983838, 0.02706, 0.0271],
 [1609195985286, 0.02706, 0.0271],
 [1609195986803, 0.02706, 0.0271],
 [1609195988331, 0.02704, 0.02708]]